In [5]:
from multiprocessing import Process, Queue, current_process, freeze_support
from queue import Empty
from aoc2024.emulator import Emulator

In [3]:
emu = Emulator()
emu.load_rom()
emu.run()

# copy the ints
target_vector = [x for x in emu.output]
print("\nThe program output is:", ','.join(map(str, target_vector)))

not enough values to unpack (expected 2, got 0)

The program output is: 2,0,4,2,7,0,1,0,3


In [7]:
NUM_PROCESSES = 2

def worker(input_queue, program_output):
    print("WORKER")
    rax = input_queue.get()
    print("RAX:", rax)
    emu = Emulator()
    emu.rax = rax
    emu.run()
    program_output.put((rax, emu.output))

input_queue = Queue()
output_queue = Queue()

rax = 0
for _ in range(NUM_PROCESSES):
    input_queue.put(rax)
    Process(target=worker, args=(input_queue, output_queue)).start()
    rax += 1
i= 0
freeze_support()
while True or i < 50000:
    i += 1
    try:
        print(output_queue.get(timeout=2))
    except Empty:
        break



'MainProcess'